In [55]:
import folium
import pandas as pd
import numpy as np

In [56]:
# Import data from GeoName
import requests
import json

def getData(place):
    base_url = 'http://api.geonames.org/searchJSON'
    payload = {'username': 'adaisp',
               'country' : 'CH',
               'q' : place,
               'fcodeName' : 'university',
                }
    response = requests.get(base_url, params=payload)
    data = json.loads(response.text)
    try:
        i = 0
        results = {
                'canton': data['geonames'][i]['adminName1'],
                'code': data['geonames'][i]['adminCode1'],
                'lat' : data['geonames'][i]['lat'],
                'long' : data['geonames'][i]['lng'],
            }
        while (data['geonames'][i]['adminName1'] == ''):
            i += 1
            results = {
                'canton': data['geonames'][i]['adminName1'],
                'code': data['geonames'][i]['adminCode1'],
                'lat' : data['geonames'][i]['lat'],
                'long' : data['geonames'][i]['lng'],
            }
    except:
        results = {
            'canton': 'No Match',
            'code': 'No Match',
            'lat' : 'No Match',
            'long' : 'No Match',
        }
        
    return results

In [57]:
grant_export = 'P3_GrantExport.csv'
raw_data = pd.read_csv(grant_export, sep=';', error_bad_lines=False)

uni_data = raw_data.loc[:, ['University', 'Approved Amount']]
uni_data[['Approved Amount']] = uni_data[['Approved Amount']].apply(pd.to_numeric, errors='coerce')

# drop the records where the field University is blank
uni_data = uni_data[uni_data.University != 'Nicht zuteilbar - NA']

# drop the records corresponding to companies or private sector
uni_data = uni_data[uni_data.University != 'Firmen/Privatwirtschaft - FP']

# drop the records corresponding to NPO
uni_data = uni_data[uni_data.University != 'NPO (Biblioth., Museen, Verwalt.) - NPO']

# we only take into account the Swiss Universities to check if we cover 95% of the records
number_valid_records = len(uni_data)

# compute the approved amount by University
uni_data = uni_data.groupby(['University'])['Approved Amount'].sum()

uni_data.sort_values(ascending=False, inplace=True)

In [58]:
data = pd.DataFrame(uni_data)
data['Records number'] = raw_data['University'].value_counts()
data['Canton'] = ['']*len(data)
data['Code'] = ['']*len(data)
data['Latitude'] = ['']*len(data)
data['Longitude'] = ['']*len(data)

for univ,row in data.iterrows():
    
    info = getData(univ.split(' - ')[0])
    
    if (info.get('canton') == 'No Match' and len(univ.split(' - ')) > 1):
        info = getData(univ.split(' - ')[1])
    
    data.set_value(univ, 'Canton', info.get('canton'))
    data.set_value(univ, 'Code', info.get('code'))
    data.set_value(univ, 'Latitude', info.get('lat'))
    data.set_value(univ, 'Longitude', info.get('long'))

After getting informations from GeoNames API, we have to check if these informations cover at least 95% of the records in the raw_data:

In [59]:
# try to look for patterns in the University name
for univ, row in data[data.index.str.contains('Luzern')].iterrows():
    data.set_value(univ, 'Canton', 'Lucerne')
    data.set_value(univ, 'Code', 'LU')
    
for univ, row in data[data.index.str.contains('Zürcher')].iterrows():
    data.set_value(univ, 'Canton', 'Zurich')
    data.set_value(univ, 'Code', 'ZH')
    
for univ, row in data[data.index.str.contains('Svizzera italiana')].iterrows():
    data.set_value(univ, 'Canton', 'Ticino')
    data.set_value(univ, 'Code', 'TI')
    
for univ, row in data[data.index.str.contains('Bern')].iterrows():
    data.set_value(univ, 'Canton', 'Bern')
    data.set_value(univ, 'Code', 'BE')

for univ, row in data[data.index.str.contains('St. Gallen')].iterrows():
    data.set_value(univ, 'Canton', 'Saint Gallen')
    data.set_value(univ, 'Code', 'SG')

In [60]:
data.head(20)

,Approved Amount,Records number,Canton,Code,Latitude,Longitude
University,,,,,,
Université de Genève - GE,1.838237e+09,6394,Geneva,GE,46.20222,6.14569
Universität Zürich - ZH,1.826843e+09,6774,Zurich,ZH,47.37092,8.53434
ETH Zürich - ETHZ,1.635597e+09,6153,Zurich,ZH,47.3763,8.54805
Universität Bern - BE,1.519373e+09,5473,Bern,BE,46.95096,7.43548
Universität Basel - BS,1.352251e+09,4746,Basel-City,BS,47.55832,7.58403
Université de Lausanne - LA,1.183291e+09,4092,Vaud,VD,46.52376,6.58409
EPF Lausanne - EPFL,1.175316e+09,4428,Vaud,VD,46.51939,6.56673
Université de Fribourg - FR,4.575262e+08,2079,Fribourg,FR,46.80683,7.15317
Université de Neuchâtel - NE,3.832046e+08,1596,Neuchâtel,NE,46.99385,6.93789


In [61]:
canton_found = data[data.Canton != 'No Match']
percentage = canton_found['Records number'].sum() / number_valid_records

percentage

0.7571243414297497

It only covers 75.43%, so we have to complete it manually.

In [62]:
univ_found = canton_found[canton_found.Latitude != 'No Match']
univ_found

,Approved Amount,Records number,Canton,Code,Latitude,Longitude
University,,,,,,
Université de Genève - GE,1.838237e+09,6394,Geneva,GE,46.20222,6.14569
Universität Zürich - ZH,1.826843e+09,6774,Zurich,ZH,47.37092,8.53434
ETH Zürich - ETHZ,1.635597e+09,6153,Zurich,ZH,47.3763,8.54805
Universität Bern - BE,1.519373e+09,5473,Bern,BE,46.95096,7.43548
Universität Basel - BS,1.352251e+09,4746,Basel-City,BS,47.55832,7.58403
Université de Lausanne - LA,1.183291e+09,4092,Vaud,VD,46.52376,6.58409
EPF Lausanne - EPFL,1.175316e+09,4428,Vaud,VD,46.51939,6.56673
Université de Fribourg - FR,4.575262e+08,2079,Fribourg,FR,46.80683,7.15317
Université de Neuchâtel - NE,3.832046e+08,1596,Neuchâtel,NE,46.99385,6.93789


In [99]:
all_cantons = np.array(['AG','AI','AR','BE','BL','BS','FR','GE','GL','GR','LU',
                        'NE','NW','OW','SG','SO','SZ','TG','TI','UR','VD','VS','ZG','ZH','SH','JU'])
amounts = canton_found.groupby(['Code'])['Approved Amount'].sum()

cantons_amount = []
for canton in all_cantons:
    if amounts.get(canton):
        cantons_amount.append(np.log10(amounts.get(canton)))
    else:
        cantons_amount.append(0)

cantons_amount_df = pd.DataFrame()
cantons_amount_df['Canton'] = all_cantons
cantons_amount_df['Amount'] = cantons_amount
cantons_amount_df.head()

,Canton,Amount
0,AG,8.062312
1,AI,0.000000
2,AR,0.000000
3,BE,9.191161
4,BL,7.631159


In [121]:
cantons_geo = 'ch-cantons.topojson.json'

swiss_map = folium.Map(location=[46.801111,8.226667], zoom_start=8)

for univ, row in univ_found.iterrows():
     folium.Marker([row['Latitude'], row['Longitude']],
                   popup=univ, 
                   icon = folium.Icon(color = 'red', icon = 'tag')).add_to(swiss_map)

swiss_map.choropleth(geo_path=cantons_geo, 
                     topojson='objects.cantons', 
                     data=cantons_amount_df,columns=['Canton', 'Amount'],
                     key_on='feature.id',
                     threshold_scale=[4, 5, 6, 7, 8, 9],
                     line_opacity=0.2, fill_opacity=0.7, fill_color='YlOrBr',
                     legend_name='Approved amount (logarithmic scale)',
                    )
swiss_map

In [123]:
swiss_map.save("map_manual.html")